In [1]:
from binance.spot import Spot
from binance.client import Client
import pandas as pd
import requests
import datetime
import numpy as np

In [2]:
%run "Binance API.ipynb".ipynb

In [3]:
binance_api_key='E90xR7UnO2oeP3lrgC50zkqUuAqyyXs4uSlAGcqIVO9n90ELrFI4gfVI2HGGxxCj'
binance_api_secret='Inls2xYl9FqPi0zlIrucfxG81drx7w6Pv6SzYFxMNoAcwDjlCKug7zUM9zB3lF9h'
Binance=BinanceAPI(binance_api_key,binance_api_secret)

In [4]:
trade_history=pd.read_excel('Trade History Reconstructed.xlsx')
trade_history['Date(UTC)']=pd.to_datetime(trade_history['Date(UTC)'])
trade_history['Time']=trade_history['Date(UTC)'].dt.round('T')
trade_history=trade_history.set_index('Date(UTC)')

trade_info=zip(trade_history['Market'],trade_history.index)
trade_info=dict(enumerate(trade_info))

trade_price={}
for index in trade_info:
    
    if trade_info[index][0][-4:]=='USDT':
        ticker=trade_info[index][0]
    else:
        ticker=trade_info[index][0][-3:]+'USDT'
        
    price=Binance.binance_api.klines(ticker,interval='1m',startTime=int(trade_info[index][1].round(freq='min').timestamp()-60)*1000,limit=1)
        
    trade_price[index]=(trade_info[index][1],trade_info[index][0],price[0][4])

In [5]:
price=pd.DataFrame(trade_price.values(),columns=['Time','Market','Pair Price'])
price=pd.concat([trade_history.reset_index(),price['Pair Price']],axis=1)
price['Price in USDT']=np.where(price['Market'].str[-4:]=='USDT',price['Price'],price['Price'].astype(float)*price['Pair Price'].astype(float))
price['Total in USDT']=price['Price in USDT']*(price['Amount'].astype(float))

In [6]:
price

,Date(UTC),Market,Type,Price,Amount,Total,Fee,Fee Coin,Time,Pair Price,Price in USDT,Total in USDT
0,2024-09-25 15:25:35,OMUSDT,BUY,1.180300,57.000000,67.277100,0.05700,OM,2024-09-25 15:26:00,1.17990000,1.180300,67.277100
1,2024-09-25 15:25:35,OMUSDT,BUY,1.180300,34.000000,40.130200,0.03400,OM,2024-09-25 15:26:00,1.17990000,1.180300,40.130200
2,2024-09-25 15:25:35,OMUSDT,BUY,1.180200,94.000000,110.938800,0.09400,OM,2024-09-25 15:26:00,1.17990000,1.180200,110.938800
3,2024-09-25 15:25:00,TRXUSDT,SELL,0.150800,1456.100000,219.579880,0.21958,USDT,2024-09-25 15:25:00,0.15080000,0.150800,219.579880
4,2024-09-18 21:45:12,OMBTC,BUY,0.000018,73.000000,0.001322,0.07300,OM,2024-09-18 21:45:00,60329.26000000,1.092563,79.757092
...,...,...,...,...,...,...,...,...,...,...,...,...
74,2024-05-09 22:32:57,TRXEUR,BUY,0.118983,205.828142,25.000000,0.51000,EUR,2024-05-09 22:33:00,1.07790000,0.128252,26.397772
75,2024-05-09 19:36:50,UMAEUR,BUY,3.803499,6.438808,25.000000,0.51000,EUR,2024-05-09 19:37:00,1.07750000,4.098270,26.387975
76,2024-05-06 23:27:11,ETHEUR,BUY,2912.268956,0.008407,25.000000,0.51000,EUR,2024-05-06 23:27:00,1.07600000,3133.601397,26.344908
77,2024-05-06 23:15:38,BTCEUR,BUY,59899.712723,0.000409,25.000000,0.51000,EUR,2024-05-06 23:16:00,1.07610000,64458.080861,26.353686


In [7]:
buy=price[price['Type']=='BUY'][['Market','Amount','Price in USDT']]

buy=buy.reset_index().drop(columns='index')
buy=buy.to_dict()

traded_crypto=set()
for keys in buy['Market']:
    traded_crypto.add(buy['Market'][keys])

average_price=dict()

In [19]:
crypto_=set()
for key in traded_crypto:
        
    if key[-4:]=='USDT':
        crypto=key[:-4]
    else:
        crypto=key[:-3]
    
    crypto_.add(crypto)


In [10]:
dynamic_average_total={}
dynamic_average_amount={}

for crypto in crypto_:

    dataset=price[price['Market'].str[:len(crypto)]==crypto]
    index=dataset[dataset['Type']=='BUY'].index
    
    results_amount=list(zip(price.iloc[index]['Date(UTC)'],price.iloc[index]['Amount']))
    results_total=list(zip(price.iloc[index]['Date(UTC)'],price.iloc[index]['Total in USDT']))
    dynamic_average_total[crypto]=results_total
    dynamic_average_amount[crypto]=results_amount
    

In [11]:
dataframe_amount={}
dataframe_total={}
for crypto in crypto_:
    temp=pd.DataFrame(dynamic_average_total[crypto],columns=['Date','Total']).groupby(by='Date').sum()
    temp_amount=pd.DataFrame(dynamic_average_amount[crypto],columns=['Date','Quantities']).groupby(by='Date').sum()
    dataframe_total[crypto+'USDT']=dict(zip(temp.index,temp['Total']))
    dataframe_amount[crypto+'USDT']=dict(zip(temp_amount.index,temp_amount['Quantities']))

In [13]:
quantities=pd.DataFrame(dataframe_amount).sort_index().cumsum().fillna(method='ffill').fillna(0)
total=pd.DataFrame(dataframe_total).sort_index().cumsum().fillna(method='ffill').fillna(0)

In [15]:
book_cost=(total.shift(-1)+total)/(quantities.shift(-1)+quantities)

In [16]:
book_cost=book_cost.fillna(0)
book_cost.iloc[-1]=total.iloc[-1]/quantities.iloc[-1]

In [17]:
book_cost.iloc[-1]

NEARUSDT         8.272700
BNBUSDT        670.860934
FETUSDT          2.292907
BBUSDT           0.516922
MKRUSDT       2786.582522
USDTUSDT         1.011770
LPTUSDT         23.209038
ETHUSDT       3616.390811
SOLUSDT        157.056383
BTCUSDT      67906.065791
TNSRUSDT         0.860996
TRXUSDT          0.158366
XRPUSDT          0.551117
RNDRUSDT        11.186443
PEPEUSDT         0.000015
OMUSDT           1.129914
UMAUSDT          4.098270
NOTUSDT          0.019619
ENSUSDT         23.729381
JASMYUSDT        0.030287
Name: 2024-09-25 15:25:35, dtype: float64

In [18]:
book_cost

,NEARUSDT,BNBUSDT,FETUSDT,BBUSDT,MKRUSDT,USDTUSDT,LPTUSDT,ETHUSDT,SOLUSDT,BTCUSDT,TNSRUSDT,TRXUSDT,XRPUSDT,RNDRUSDT,PEPEUSDT,OMUSDT,UMAUSDT,NOTUSDT,ENSUSDT,JASMYUSDT
2024-05-02 22:48:24,0.0000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,140.487792,64458.080861,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000
2024-05-06 23:15:38,0.0000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,3133.601397,140.487792,64458.080861,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000
2024-05-06 23:27:11,0.0000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,3133.601397,140.487792,64458.080861,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,4.09827,0.000000,0.000000,0.000000
2024-05-09 19:36:50,0.0000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,3133.601397,140.487792,64458.080861,0.000000,0.128252,0.000000,0.000000,0.000000,0.000000,4.09827,0.000000,0.000000,0.000000
2024-05-09 22:32:57,0.0000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,3133.601397,140.487792,64458.080861,0.000000,0.128252,0.000000,11.029465,0.000000,0.000000,4.09827,0.000000,0.000000,0.000000
2024-05-10 00:25:59,0.0000,0.000000,2.309730,0.000000,0.000000,0.00000,0.000000,3133.601397,140.487792,64458.080861,0.000000,0.128252,0.000000,11.029465,0.000000,0.000000,4.09827,0.000000,0.000000,0.000000
2024-05-10 01:20:55,0.0000,0.000000,2.309730,0.000000,2784.378089,0.00000,0.000000,3133.601397,140.487792,64458.080861,0.000000,0.128252,0.000000,11.029465,0.000000,0.000000,4.09827,0.000000,0.000000,0.000000
2024-05-10 01:40:37,0.0000,0.000000,2.309730,0.000000,2784.378089,1.01177,0.000000,3133.601397,140.487792,64458.080861,0.000000,0.128252,0.000000,11.029465,0.000000,0.000000,4.09827,0.000000,0.000000,0.000000
2024-05-10 01:42:14,0.0000,0.000000,2.309730,0.000000,2784.378089,1.01177,0.000000,3133.601397,144.352276,64458.080861,0.000000,0.128252,0.000000,11.029465,0.000000,0.000000,4.09827,0.000000,0.000000,0.000000
2024-05-10 01:58:23,0.0000,0.000000,2.309730,0.000000,2784.378089,1.01177,0.000000,3133.601397,146.585596,64458.080861,0.860996,0.128252,0.000000,11.029465,0.000000,0.000000,4.09827,0.000000,0.000000,0.000000
